### Check different merits of the model

In [1]:
import os
import numpy as np
from copy import deepcopy

prefix = os.getcwd()

In [2]:
def apply_rank_model(X_train,Y_train,X_test,Y_test,BaseNames):
    """
    Apply the ranking SVM model and get merits we need for analysis
    """
    feature_comp = ['(cbrt(NNaLst)+(AnionChgStdLst)^2)', '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))']
    
    RankedFeatTrain,RankedTargetTrain,RankedFeatTest,RankedTargetTest = X_train,Y_train,X_test,Y_test
    TrainFeatureMat=__getFeatureMat(X_train,BaseNames,feature_comp)
    TestFeatureMat=__getFeatureMat(X_test,BaseNames,feature_comp)
    print(TrainFeatureMat.shape, TestFeatureMat.shape)
    TransTrainFeatMat,TransTrainTargets=rankedTransform(TrainFeatureMat,RankedTargetTrain);
    TransTestFeatMat,TransTestTargets=rankedTransform(TestFeatureMat,RankedTargetTest);
    print(np.unique(TransTrainTargets))
    print(np.unique(TransTestTargets))
    Clf=LinearSVC(penalty='l2',loss='squared_hinge',dual=False,tol=0.0001,C=1.0,multi_class='ovr',\
              fit_intercept=True,intercept_scaling=1,class_weight=None,verbose=0,\
              random_state=None,max_iter=1000)
    
    Clf.fit(TransTrainFeatMat,TransTrainTargets);
    Y_test_predict=Clf.predict(TransTestFeatMat);
    Y_train_predict=Clf.predict(TransTrainFeatMat);
    TrainScore=accuracy_score(TransTrainTargets,Y_train_predict);
    Score=accuracy_score(TransTestTargets,Y_test_predict);
    F1Train=f1_score(TransTrainTargets,Y_train_predict);
    F1Test=f1_score(TransTestTargets,Y_test_predict);
    F1TrainVals=f1_score(TransTrainTargets,Y_train_predict,average=None);
    F1TestVals=f1_score(TransTestTargets,Y_test_predict,average=None);
    
    print('Feature comp index has CV score {}'.format(Score));
    print('Feature comp index has train score {}'.format(TrainScore));
    print('Feature comp index has CV F1 {}'.format(F1Test));
    print('Feature comp index has train F1 {}'.format(F1Train));
    
    return Score, F1Test, TransTrainFeatMat, TransTrainTargets

def __getfeatureNames(Features):
    '''
    Get the feature names
    '''
    FeatureNames=[];
    for Feature in Features:
        for Name,Feat in globals().items():
            if Feat is Feature: FeatureNames.append(Name); break;
    return FeatureNames;  

In [3]:
train_data_file = '/projects/saepssic/bouyang/NASICONProject/NASICON/SISSO/Stability/RegforAllround2/train.dat'
test_data_file = '/projects/saepssic/bouyang/NASICONProject/NASICON/SISSO/Stability/RegforAllround2/predict.dat'

In [4]:
def get_data(data_name):
    """
    Transfer train.dat and predict.dat
    """
    data_lsts = []
    with open(data_name,'r') as fid:
        lines = fid.read().split('\n')
    print(lines[0])
    
    for line in lines[1:-1]:
        data_lsts.append([eval(string) for string in line.split(' ')[1:]])
    return np.array(data_lsts)

In [5]:
train_data = get_data(train_data_file)
print(train_data.shape)
test_data = get_data(test_data_file)
print(test_data.shape)

materials property EWaldSumLst EFormLst RDiffLst RDiffStdLst RDiffNaLst EleNegStdLst XDiffLst AnionXStdLst XWithNaLst ChgDiffLst ChgStdLst AnionChgStdLst NNaLst RRatioNaLst
(3079, 15)
materials property EWaldSumLst EFormLst RDiffLst RDiffStdLst RDiffNaLst EleNegStdLst XDiffLst AnionXStdLst XWithNaLst ChgDiffLst ChgStdLst AnionChgStdLst NNaLst RRatioNaLst
(771, 15)


### Regenerate the ranking SVM model merits

In [6]:
feature_comp = ['(cbrt(NNaLst)+(AnionChgStdLst)^2)', '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))']

In [7]:
import sys
sys.path.append('/home/bouyang/Software/Nanokit/Python/PythonScript')
from SISSOAna import __readdata, __getFeatureMat, rankedTransform
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

In [8]:
X_t,Y_t,BaseNames=__readdata(train_data_file,GetNames=True,RegtoClass=1,ThredVal=0.0);
X_test,Y_test=__readdata(test_data_file,GetNames=False,RegtoClass=1,ThredVal=0.0);

In [9]:
Score, F1, _, _ = apply_rank_model(X_t,Y_t,X_test,Y_test,BaseNames)

(3079, 2) (771, 2)
NewFeatMat has the shape (4738581, 2)
NewTarget has the shape (4738581,)
NewFeatMat has the shape (296835, 2)
NewTarget has the shape (296835,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470766587498105
Feature comp index has train score 0.8433896138949614
Feature comp index has CV F1 0.7240429683937918
Feature comp index has train F1 0.7205161317535442


In [10]:
from monty.serialization import loadfn
import numpy as np

EhullInfoDictAll=loadfn('EhullInfoDictAll.mson');
EhullLst=[]; EWaldSumLst=[]; EFormLst=[]; CompNameLst=[]; NNaLst=[]; 
AvgRadiusLst=[]; RDiffLst=[]; RDiffStdLst=[]; RDiffNaLst=[]; RRatioNaLst=[];
AvgEleNegLst=[]; EleNegStdLst=[]; XDiffLst=[]; ChgDiffLst=[]; ChgStdLst=[]; AvgChgLst=[];
AnionChgDiffLst=[]; AnionChgStdLst=[]; AnionAvgChgLst=[]; AvgAnionRadiusLst=[]; AnionRDiffLst=[]; AnionRStdLst=[]; 
AvgAnionEleNegLst=[]; AnionXDiffLst=[]; AnionXStdLst=[]; XWithNaLst=[]; XWithAnionLst=[]; G_MaxLst=[];

for CompName,InfoDict in EhullInfoDictAll.items():
    if InfoDict['Ehull']<=0.3 and InfoDict['Ehull']>-0.1: continue;
    #if InfoDict['NNa']<=1: continue;
    EhullLst.append(InfoDict['Ehull']*1000-InfoDict['G_Max']); EWaldSumLst.append(InfoDict['EWaldSum']); NNaLst.append(InfoDict['NNa']);
    AvgRadiusLst.append(InfoDict['AvgRadius']);RDiffLst.append(InfoDict['RDiff']);RDiffStdLst.append(InfoDict['RStd']);
    RDiffNaLst.append(InfoDict['RDiffNa']); RRatioNaLst.append(InfoDict['RRatioNa']);
    EFormLst.append(InfoDict['ECompForm']);
    AvgEleNegLst.append(InfoDict['AvgEleNeg']);XDiffLst.append(InfoDict['XDiff']);EleNegStdLst.append(InfoDict['XStd']);
    ChgDiffLst.append(InfoDict['ChgDiff']);ChgStdLst.append(InfoDict['ChgStd']);AvgChgLst.append(InfoDict['AvgChg'])
    CompNameLst.append(CompName);
    AnionChgDiffLst.append(InfoDict['AnionChgDiff']); AnionChgStdLst.append(InfoDict['AnionChgStd']);
    AnionAvgChgLst.append(InfoDict['AnionAvgChg'])
    AvgAnionRadiusLst.append(InfoDict['AvgAnionRadius']); AnionRDiffLst.append(InfoDict['AnionRDiff']);
    AnionRStdLst.append(InfoDict['AnionRStd']);
    AvgAnionEleNegLst.append(InfoDict['AvgAnionEleNeg']); AnionXDiffLst.append(InfoDict['AnionXDiff']);
    AnionXStdLst.append(InfoDict['AnionXStd']);
    XWithNaLst.append(InfoDict['XWithNa']); XWithAnionLst.append(InfoDict['XWithAnion']);
    G_MaxLst.append(InfoDict['G_Max']);

NewEhullLst=deepcopy(EhullLst)

In [11]:
print(len(NewEhullLst))
print(NewEhullLst)

31
[-126.9368414450095, -149.5812041901138, -118.38588556265928, -223.92725490195352, -129.67768971927256, -127.71158980361642, -142.3241848173133, -120.23992011143164, -131.59297710936377, -163.34498388775333, -120.76495928891151, -133.01134719476164, -126.11663882321845, -134.54055123886948, -132.80383174278762, 489.0633246821604, 7213.056708295231, 7831.453970356243, 7589.071192426387, 7773.240163216455, 7727.222464498504, 7972.829847605548, 7849.268022776661, 2125.710688494934, 7335.277958162416, 7083.590611950638, 7563.131637089584, 7102.6951700327345, 6912.1901424536845, 7402.426697626763, 7475.248581899021]


In [12]:
Features=[NewEhullLst,EWaldSumLst,EFormLst,\
          AvgRadiusLst,RDiffLst,RDiffStdLst,RDiffNaLst,AvgAnionRadiusLst,AnionRDiffLst,AnionRStdLst,\
          G_MaxLst,\
          AvgEleNegLst,EleNegStdLst,XDiffLst,AvgAnionEleNegLst,AnionXDiffLst,AnionXStdLst,XWithNaLst,XWithAnionLst,\
          ChgDiffLst,ChgStdLst,AvgChgLst,\
          NNaLst,RRatioNaLst,\
          AnionChgDiffLst,AnionChgStdLst,AnionAvgChgLst];
KeyFeatInds=[0,1,2,4,5,6,12,13,16,17,19,20,25,22,23];
KeyFeatures=[Features[Ind] for Ind in KeyFeatInds];

FeatureNames=__getfeatureNames(Features);
KeyFeatureNames=[FeatureNames[Ind] for Ind in KeyFeatInds]
print(BaseNames)
print(KeyFeatureNames)

['EWaldSumLst', 'EFormLst', 'RDiffLst', 'RDiffStdLst', 'RDiffNaLst', 'EleNegStdLst', 'XDiffLst', 'AnionXStdLst', 'XWithNaLst', 'ChgDiffLst', 'ChgStdLst', 'AnionChgStdLst', 'NNaLst', 'RRatioNaLst']
['NewEhullLst', 'EWaldSumLst', 'EFormLst', 'RDiffLst', 'RDiffStdLst', 'RDiffNaLst', 'EleNegStdLst', 'XDiffLst', 'AnionXStdLst', 'XWithNaLst', 'ChgDiffLst', 'ChgStdLst', 'AnionChgStdLst', 'NNaLst', 'RRatioNaLst']


In [13]:
nrow, ncol = np.array(KeyFeatures).shape
Outliers = np.zeros([ncol,nrow])

for i, feat_vals in enumerate(KeyFeatures):
    for j, val in enumerate(feat_vals):
        Outliers[j,i] = val

print(Outliers.shape)

(31, 15)


In [14]:
print(train_data.shape)

(3079, 15)


In [15]:
min_delta = 1e10
for i in range(100):
    print(f'========================================')
    print(f'performing loop {i}')

    moretraininds = np.random.choice(Outliers.shape[0],size=25,replace=False)
    moretestinds = [ind for ind in range(Outliers.shape[0]) if ind not in moretraininds]

    more_train_data = np.array([Outliers[j,:] for j in moretraininds])
    more_test_data = np.array([Outliers[j,:] for j in moretestinds])

    print(train_data.shape,more_train_data.shape)
    print(test_data.shape,more_test_data.shape)

    new_train_data = np.concatenate((train_data,more_train_data))
    new_test_data = np.concatenate((test_data,more_test_data))
    print(new_train_data.shape)
    print(new_test_data.shape)

    new_X_train, new_Y_train, new_X_test, new_Y_test = new_train_data[:,1:], \
            new_train_data[:,0], new_test_data[:,1:], new_test_data[:,0]

    print(new_X_train.shape,new_Y_train.shape,new_X_test.shape,new_Y_test.shape)

    score, F1, _, _ = apply_rank_model(new_X_train, new_Y_train, new_X_test, new_Y_test, BaseNames)
    
    delta_score = abs(score-0.847)
    delta_F1 = abs(F1-0.723)
    
    if delta_score + delta_F1 < min_delta:
        final_train_inds = moretraininds
        final_test_inds = moretestinds
        min_delta = delta_score
        print(f'min_delta = {min_delta}')
        
    print(f'========================================')

performing loop 0
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8466080218657538
Feature comp index has train score 0.8418069394101485
Feature comp index has CV F1 0.7221581350636868
Feature comp index has train F1 0.720337107152808
min_delta = 0.00039197813424618477
performing loop 1
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8442164550411974
Feature comp index has train score 0.8424552146077458
Feature comp index has CV F1 0.7222994187593498
Feature comp inde

NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167
min_delta = 9.230585519248713e-05
performing loop 15
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8472316204274967
Feature comp index has train score 0.8416279473472629
Feature comp index has CV F1 0.7232444385688706
Feature comp index has train F1 0.7200200579050869
performing loop 16
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (77

NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8457422813092916
Feature comp index has train score 0.8420197780000066
Feature comp index has CV F1 0.7235547629662654
Feature comp index has train F1 0.7200922417859116
performing loop 29
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8442993803818546
Feature comp index has train score 0.8424217833755827
Feature comp index has CV F1 0.7220643266543508
Feature comp index has train F1 0.7209459847057421
performing loop 30
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2

Feature comp index has CV score 0.8463327097347716
Feature comp index has train score 0.8418947742623534
Feature comp index has CV F1 0.7249594803991998
Feature comp index has train F1 0.7199820238206441
performing loop 43
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8443856227361382
Feature comp index has train score 0.8423825380160869
Feature comp index has CV F1 0.722293913599394
Feature comp index has train F1 0.7209088131446555
performing loop 44
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]

NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8476628321989147
Feature comp index has train score 0.8415405277898674
Feature comp index has CV F1 0.7260112158453645
Feature comp index has train F1 0.7191579722145551
performing loop 58
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8442330401093288
Feature comp index has train score 0.8424329963354386
Feature comp index has CV F1 0.7172515112834469
Feature comp index has train F1 0.7223055787734047
performing loop 59
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2

NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8462000291897199
Feature comp index has train score 0.8419161619450416
Feature comp index has CV F1 0.7218653085953198
Feature comp index has train F1 0.7205978906913185
performing loop 72
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8452679483607318
Feature comp index has train score 0.8421942018199879
Feature comp index has CV F1 0.7233050596120767
Feature comp index has train F1 0.7206738201494748
performing loop 73
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2

Feature comp index has CV score 0.8451518528838117
Feature comp index has train score 0.8422004312421302
Feature comp index has CV F1 0.7234796206677999
Feature comp index has train F1 0.7205724892191296
performing loop 86
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8419907388979554
Feature comp index has train score 0.8429872072586888
Feature comp index has CV F1 0.7151502104860316
Feature comp index has train F1 0.7234999669070689
performing loop 87
(3079, 15) (25, 15)
(771, 15) (6, 15)
(3104, 15)
(777, 15)
(3104, 14) (3104,) (777, 14) (777,)
(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.

In [16]:
moretraininds = final_train_inds
moretestinds = final_test_inds 

more_train_data = np.array([Outliers[ind,:] for ind in moretraininds])
more_test_data = np.array([Outliers[ind,:] for ind in moretestinds])

out_train_data = np.concatenate((train_data,more_train_data))
out_test_data = np.concatenate((test_data,more_test_data))

out_X_train, out_Y_train, out_X_test, out_Y_test = out_train_data[:,1:], \
            out_train_data[:,0], out_test_data[:,1:], out_test_data[:,0]

score, F1, _, _ = apply_rank_model(out_X_train, out_Y_train, out_X_test, out_Y_test, BaseNames)

(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167


In [17]:
print(out_X_train.shape)

(3104, 14)


## Process the output of the data

### Note!!!!!!
The ranked SVM has the feature that the order of the data should be exactly the same, otherwise it will give different merit

In [18]:
Out_Y_train_lst = out_Y_train.tolist()
Out_Y_test_lst = out_Y_test.tolist()
count_train, count_test = 0, 0
train_ind_visited, test_ind_visited = [], []

for CompName,InfoDict in EhullInfoDictAll.items():
    ehull_val = InfoDict['Ehull']*1000 - InfoDict['G_Max']
    
    if ehull_val in Out_Y_train_lst:
        print(f'{CompName} is in training set with indice {Out_Y_train_lst.index(ehull_val)} and ehull {ehull_val}')
        chk_ind = Out_Y_train_lst.index(ehull_val)
        if chk_ind not in train_ind_visited:
            train_ind_visited.append(chk_ind)
        else:
            print(f'{CompName} has duplicated ehull value')
        count_train += 1
    else:
        print(f'{CompName} is in testing set with indice {Out_Y_test_lst.index(ehull_val)} and ehull {ehull_val}')
        chk_ind = Out_Y_test_lst.index(ehull_val)
        if chk_ind not in test_ind_visited:
            test_ind_visited.append(chk_ind)
        else:
            print(f'{CompName} has duplicated ehull value')
        count_test += 1

print(f'count_train = {count_train}, count_test = {count_test}')

MgZr(SO4)3 is in training set with indice 0 and ehull -60.26056693521266
MgTi(SO4)3 is in training set with indice 1 and ehull -27.986571837184684
MgSn(SO4)3 is in training set with indice 2 and ehull -30.64096644501392
MgGe(SO4)3 is in testing set with indice 0 and ehull -9.21553752344674
HfMg(SO4)3 is in training set with indice 3086 and ehull -126.9368414450095
Mg4Nb2(SO4)9 is in training set with indice 3 and ehull -42.78312693527414
Mg4Ta2(SO4)9 is in testing set with indice 1 and ehull -48.21565907035002
ZrZn(SO4)3 is in training set with indice 4 and ehull -40.419096346974506
TiZn(SO4)3 is in training set with indice 5 and ehull -33.03105713129252
ZnSn(SO4)3 is in testing set with indice 2 and ehull -21.22907183718871
ZnGe(SO4)3 is in testing set with indice 3 and ehull -25.113429680310286
HfZn(SO4)3 is in training set with indice 3098 and ehull -149.5812041901138
Nb2Zn4(SO4)9 is in training set with indice 6 and ehull -68.29845237101448
Ta2Zn4(SO4)9 is in training set with indi

Na3Zr3InSi3(SO8)3 is in training set with indice 1310 and ehull 47.43300484411847
Na3Zr3In(PO4)6 is in training set with indice 1311 and ehull 0.11319330477839173
Na3Ti3InSi3(SO8)3 is in training set with indice 1312 and ehull 51.06957579007073
Na3Ti3In(PO4)6 is in training set with indice 1313 and ehull 3.1442293408085575
Na3InSi3Sn3(SO8)3 is in training set with indice 1314 and ehull 90.77175146574598
Na3InSn3(PO4)6 is in training set with indice 1315 and ehull 3.6164590705480393
Na3InSi3Ge3(SO8)3 is in training set with indice 1316 and ehull 126.74439110538268
Na3InGe3(PO4)6 is in training set with indice 1317 and ehull 26.583965827298563
Na3Hf3InSi3(SO8)3 is in training set with indice 1318 and ehull 37.12219403330417
Na3Hf3In(PO4)6 is in training set with indice 1319 and ehull 1.4103554669413043
Na9Nb4.5In7.5Si9S9O72 is in testing set with indice 335 and ehull 64.37702215805007
Na9Nb4.5In7.5P18O72 is in training set with indice 1320 and ehull 2.7851892223034955
Na9Ta4.5In7.5Si9S9O

Na15Mg9Zr3Si4.5S13.5O72 is in training set with indice 1754 and ehull 51.97011226751033
Na5Mg3TiP3(SO8)3 is in testing set with indice 446 and ehull 17.25879235271207
Na15Mg9Ti3Si4.5S13.5O72 is in training set with indice 1755 and ehull 52.18871403033573
Na5Mg3SnP3(SO8)3 is in training set with indice 1756 and ehull 20.44231799372472
Na15Mg9Si4.5Sn3S13.5O72 is in training set with indice 1757 and ehull 60.37689538717281
Na5Mg3GeP3(SO8)3 is in testing set with indice 447 and ehull 26.95618551509967
Na15Mg9Si4.5Ge3S13.5O72 is in testing set with indice 448 and ehull 81.15053694698157
Na5HfMg3P3(SO8)3 is in testing set with indice 449 and ehull 21.841954745879654
Na15Hf3Mg9Si4.5S13.5O72 is in training set with indice 1758 and ehull 50.18071055811211
Na15Mg10Nb2P9(SO8)9 is in training set with indice 1759 and ehull 9.443414316086844
Na15Mg10Nb2Si4.5S13.5O72 is in training set with indice 1760 and ehull 52.32887685907539
Na15Mg10Ta2P9(SO8)9 is in training set with indice 1761 and ehull 10.3

Na3Sc2(PO4)3 is in training set with indice 2288 and ehull 1.5239560171250872
Na6Y2Sc2Si3(SO8)3 is in training set with indice 2289 and ehull 60.48063200331044
Na3YSc(PO4)3 is in training set with indice 460 and ehull -1.0171698383999797
Na6Sc2In2Si3(SO8)3 is in training set with indice 2290 and ehull 71.1131903366493
Na3ScIn(PO4)3 is in training set with indice 2291 and ehull 1.5325557682664623
Na6La2Sc2Si3(SO8)3 is in training set with indice 2292 and ehull 77.76606950330466
Na3LaSc(PO4)3 is in training set with indice 2293 and ehull 29.764825994929986
Na6Y4Si3(SO8)3 is in testing set with indice 580 and ehull 75.05922973383488
Na3Y2(PO4)3 is in training set with indice 2294 and ehull 4.564756017124598
Na6Y2In2Si3(SO8)3 is in training set with indice 2295 and ehull 82.40761533664931
Na3YIn(PO4)3 is in training set with indice 2296 and ehull 4.8531468282705035
Na6La2Y2Si3(SO8)3 is in training set with indice 2297 and ehull 74.80259971164728
Na3LaY(PO4)3 is in training set with indice 

Na7Ca2Y2P5SO24 is in testing set with indice 636 and ehull 10.034441967638084
Na21Ca6Y6Si7.5S10.5O72 is in training set with indice 2523 and ehull 73.09348007239727
Na7Ca2In2P5SO24 is in training set with indice 2524 and ehull 20.640515138367675
Na21Ca6In6Si7.5S10.5O72 is in training set with indice 2525 and ehull 86.34099277564636
Na7Ca2La2P5SO24 is in training set with indice 2526 and ehull 28.780698065190066
Na21Ca6La6Si7.5S10.5O72 is in training set with indice 2527 and ehull 83.69074633255622
Na7Ca3ZrP5SO24 is in training set with indice 2528 and ehull 8.183378689325089
Na21Ca9Zr3Si7.5S10.5O72 is in training set with indice 2529 and ehull 78.69876842009714
Na7Ca3TiP5SO24 is in training set with indice 2530 and ehull 21.832893594478904
Na21Ca9Ti3Si7.5S10.5O72 is in training set with indice 2531 and ehull 86.36754382660067
Na7Ca3SnP5SO24 is in training set with indice 2532 and ehull 20.271767191073238
Na21Ca9Si7.5Sn3S10.5O72 is in training set with indice 2533 and ehull 112.19850622

Na8La3TaSi4(PO12)2 is in training set with indice 3056 and ehull 59.112785895029305
Na12Mg2Nb4(SiO4)9 is in training set with indice 3057 and ehull 45.049615878148984
Na12Mg2Ta4(SiO4)9 is in training set with indice 3058 and ehull 34.49363175117032
Na12Nb4Zn2(SiO4)9 is in testing set with indice 764 and ehull 61.0628301638721
Na12Ta4Zn2(SiO4)9 is in training set with indice 3059 and ehull 51.069147624189846
Na12Ca2Nb4(SiO4)9 is in training set with indice 3060 and ehull 58.543129105666786
Na12Ca2Ta4(SiO4)9 is in training set with indice 3061 and ehull 56.46404974058411
Na4NbGa(SiO4)3 is in training set with indice 3062 and ehull 46.74957974756176
Na4TaGa(SiO4)3 is in testing set with indice 765 and ehull 39.216063874546776
Na4NbAl(SiO4)3 is in training set with indice 3063 and ehull 45.622516255499605
Na4TaAl(SiO4)3 is in training set with indice 3064 and ehull 37.850825779307854
Na4ScNb(SiO4)3 is in training set with indice 3065 and ehull 27.16880990628999
Na4ScTa(SiO4)3 is in trainin

In [19]:
ColumnNames = ['CompName'] + KeyFeatureNames
print(len(ColumnNames))
print(out_X_train.shape)

16
(3104, 14)


In [20]:
Out_Y_train_lst = out_Y_train.tolist()
Out_Y_test_lst = out_Y_test.tolist()

train_dict = {key: [] for key in ColumnNames}
test_dict = {key: [] for key in ColumnNames}

for CompName,InfoDict in EhullInfoDictAll.items():
    ehull_val = InfoDict['Ehull']*1000 - InfoDict['G_Max']
#     print(out_X_train.shape)
    
    if ehull_val in Out_Y_train_lst:
        ehull_ind = Out_Y_train_lst.index(ehull_val)
        train_dict['CompName'].append(CompName)
        train_dict[KeyFeatureNames[0]].append(ehull_val)
        val_lst = []
        for i, feat_name in enumerate(KeyFeatureNames):
            if i==0: 
                continue
            train_dict[feat_name].append(out_X_train[ehull_ind,i-1])
            val_lst.append(out_X_train[ehull_ind,i-1])
        diff_sum = np.sum(np.array(val_lst)-out_X_train[ehull_ind])
        if diff_sum != 0:
            print(diff_sum)

    elif ehull_val in Out_Y_test_lst:
        ehull_ind = Out_Y_test_lst.index(ehull_val)
        test_dict['CompName'].append(CompName)
        test_dict[KeyFeatureNames[0]].append(ehull_val)
        val_lst = []
        for i, feat_name in enumerate(KeyFeatureNames):
            if i==0: continue
            test_dict[feat_name].append(out_X_test[ehull_ind,i-1])
            val_lst.append(out_X_test[ehull_ind,i-1])
        diff_sum = np.sum(np.array(val_lst)-out_X_test[ehull_ind])
        if diff_sum != 0:
            print(diff_sum)
    
    else:
        raise Exception(f'Unkown value {ehull_val}')

In [21]:
# ['EWaldSumLst', 'EFormLst', 'RDiffLst', 'RDiffStdLst', 'RDiffNaLst', 'EleNegStdLst', 'XDiffLst', 'AnionXStdLst', 'XWithNaLst', 'ChgDiffLst', 'ChgStdLst', 'AnionChgStdLst', 'NNaLst', 'RRatioNaLst']
print(KeyFeatureNames)

['NewEhullLst', 'EWaldSumLst', 'EFormLst', 'RDiffLst', 'RDiffStdLst', 'RDiffNaLst', 'EleNegStdLst', 'XDiffLst', 'AnionXStdLst', 'XWithNaLst', 'ChgDiffLst', 'ChgStdLst', 'AnionChgStdLst', 'NNaLst', 'RRatioNaLst']


In [22]:
from pandas import DataFrame

train_df = DataFrame.from_dict(train_dict)
test_df = DataFrame.from_dict(test_dict)

In [23]:
print(train_df.head())
print(train_df.shape)
print(out_X_train[:5,:])

       CompName  NewEhullLst  EWaldSumLst  EFormLst  RDiffLst  RDiffStdLst  \
0    MgZr(SO4)3   -60.260567   -65.921154 -1.891755     0.000     0.000000   
1    MgTi(SO4)3   -27.986572   -66.524285 -1.828661     0.115     0.057500   
2    MgSn(SO4)3   -30.640966   -66.032051 -1.554568     0.030     0.015000   
3    HfMg(SO4)3  -126.936841   -66.110447 -1.863254     0.010     0.005000   
4  Mg4Nb2(SO4)9   -42.783127   -66.565193 -1.750108     0.080     0.037712   

   RDiffNaLst  EleNegStdLst  XDiffLst  AnionXStdLst  XWithNaLst  ChgDiffLst  \
0   -0.300000      0.010000      0.02           0.0    0.390000         2.0   
1   -0.357500      0.115000      0.23           0.0    0.495000         2.0   
2   -0.315000      0.325000      0.65           0.0    0.705000         2.0   
3   -0.305000      0.005000      0.01           0.0    0.375000         2.0   
4   -0.326667      0.136707      0.29           0.0    0.476667         3.0   

   ChgStdLst  AnionChgStdLst  NNaLst  RRatioNaLst  
0   

In [24]:
test_df.head()
print(test_df.shape)

(777, 16)


## Sanity check for output data

In [25]:
print(train_df.head())

       CompName  NewEhullLst  EWaldSumLst  EFormLst  RDiffLst  RDiffStdLst  \
0    MgZr(SO4)3   -60.260567   -65.921154 -1.891755     0.000     0.000000   
1    MgTi(SO4)3   -27.986572   -66.524285 -1.828661     0.115     0.057500   
2    MgSn(SO4)3   -30.640966   -66.032051 -1.554568     0.030     0.015000   
3    HfMg(SO4)3  -126.936841   -66.110447 -1.863254     0.010     0.005000   
4  Mg4Nb2(SO4)9   -42.783127   -66.565193 -1.750108     0.080     0.037712   

   RDiffNaLst  EleNegStdLst  XDiffLst  AnionXStdLst  XWithNaLst  ChgDiffLst  \
0   -0.300000      0.010000      0.02           0.0    0.390000         2.0   
1   -0.357500      0.115000      0.23           0.0    0.495000         2.0   
2   -0.315000      0.325000      0.65           0.0    0.705000         2.0   
3   -0.305000      0.005000      0.01           0.0    0.375000         2.0   
4   -0.326667      0.136707      0.29           0.0    0.476667         3.0   

   ChgStdLst  AnionChgStdLst  NNaLst  RRatioNaLst  
0   

In [26]:
sanity_X_train, sanity_Y_train, sanity_X_test, sanity_Y_test = train_df.to_numpy()[:,2:].astype('float64'), \
                                                   train_df.to_numpy()[:,1].astype('float64'), \
                                                   test_df.to_numpy()[:,2:].astype('float64'), \
                                                   test_df.to_numpy()[:,1].astype('float64')
print(out_X_train[:1,-2])
np.cbrt(out_X_train[:,-2])

[0.]


array([0.        , 0.        , 0.        , ..., 1.51829449, 1.35720881,
       0.        ])

In [27]:
print(np.sum(sanity_X_train,axis=0))
print(np.sum(out_X_train,axis=0))
print(np.sum(sanity_X_train-out_X_train))

print(np.sum(sanity_X_test,axis=0))
print(np.sum(out_X_test,axis=0))
print(np.sum(sanity_X_test-out_X_test))

print(np.sum(sanity_Y_train,axis=0))
print(np.sum(out_Y_train,axis=0))
print(np.sum(sanity_Y_train-out_Y_train))

print(np.sum(sanity_Y_test,axis=0))
print(np.sum(out_Y_test,axis=0))
print(np.sum(sanity_Y_test-out_Y_test))

[-157935.82646656   -7123.36459621     491.846          228.95660526
    -918.00458333     465.9669841      996.77           598.94418328
    1767.10666667    4608.            2094.01633716    1757.78356355
    6321.5           2203.99550654]
[-157935.82646656   -7123.36459621     491.846          228.95660526
    -918.00458333     465.9669841      996.77           598.94418328
    1767.10666667    4608.            2094.01633716    1757.78356355
    6321.5           2203.99550654]
1.7053025658242404e-13
[-39508.99262117  -1779.11662152    125.43           58.67836154
   -229.89691667    114.45083912    244.89          150.83782012
    443.00333333   1172.            535.88452912    441.33806128
   1587.5           551.61086601]
[-39508.99262117  -1779.11662152    125.43           58.67836154
   -229.89691667    114.45083912    244.89          150.83782012
    443.00333333   1172.            535.88452912    441.33806128
   1587.5           551.61086601]
2.842170943040401e-14
194337.6131

In [28]:
score, F1, sanity_FeatMat, sanity_Targets = \
    apply_rank_model(sanity_X_train, sanity_Y_train, sanity_X_test, sanity_Y_test, BaseNames)

(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8400735050219587
Feature comp index has train score 0.8379218564674691
Feature comp index has CV F1 0.764926719388402
Feature comp index has train F1 0.759000823456436


In [29]:
score, F1, out_FeatMat, out_Targets = \
    apply_rank_model(out_X_train, out_Y_train, out_X_test, out_Y_test, BaseNames)

(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167


In [30]:
print(np.sum(sanity_FeatMat,axis=0))
print(np.sum(out_FeatMat,axis=0))

[-2.03707586e+06  1.45373628e+08]
[-1955743.78112286 94461951.35610679]


In [31]:
print(np.where(train_df['EWaldSumLst']==-46.4490377))
np.sort(np.abs(train_df['EWaldSumLst'].to_numpy()--46.4490377))

(array([], dtype=int64),)


array([4.39798242e-09, 3.44797739e-04, 2.15146855e-03, ...,
       2.01194067e+01, 2.04238538e+01, 3.64643401e+01])

### Adjust the order and recheck

In [32]:
new_sanity_X_train_lst, new_sanity_Y_train_lst = [], []
out_ehull_train_lst = [val for val in sanity_Y_train]

new_inds_train, new_inds_test = [], []
new_train_dict, new_test_dict = {col_name: [] for col_name in train_dict.keys()}, \
                                {col_name: [] for col_name in test_dict.keys()}

visited_train_dict = {}
for data_i, data_arry in enumerate(out_Y_train):
    ehull_ind = out_ehull_train_lst.index(data_arry)
    if ehull_ind in visited_train_dict:
        print(f'{ehull_ind} visited')
        print(visited_train_dict[ehull_ind], data_i)
        print(out_X_train[data_i,:])
        print(out_X_train[visited_train_dict[ehull_ind],:])
    else:
        visited_train_dict[ehull_ind] = data_i
    new_sanity_X_train_lst.append(sanity_X_train[ehull_ind,:])
    new_sanity_Y_train_lst.append(sanity_Y_train[ehull_ind])
    new_inds_train.append(ehull_ind)
    for col_name in train_dict.keys():
        new_train_dict[col_name].append(train_dict[col_name][ehull_ind])

new_sanity_X_test_lst, new_sanity_Y_test_lst = [], []
out_ehull_test_lst = [val for val in sanity_Y_test]

visited_test_dict = {}
for data_i, data_arry in enumerate(out_Y_test):
    ehull_ind = out_ehull_test_lst.index(data_arry)
    if ehull_ind in visited_test_dict:
        print(f'{ehull_ind} visited')
        print(visited_test_dict[ehull_ind], data_i)
        print(print(out_X_test[data_i,:]))
        print(out_X_test[visited_test_dict[ehull_ind],:])
    else:
        visited_test_dict[ehull_ind] = data_i
    new_sanity_X_test_lst.append(sanity_X_test[ehull_ind,:])
    new_sanity_Y_test_lst.append(sanity_Y_test[ehull_ind])
    new_inds_test.append(ehull_ind)
    for col_name in test_dict.keys():
        new_test_dict[col_name].append(test_dict[col_name][ehull_ind])

new_sanity_X_train, new_sanity_Y_train, new_sanity_X_test, new_sanity_Y_test = \
    np.array(new_sanity_X_train_lst), np.array(new_sanity_Y_train_lst), \
    np.array(new_sanity_X_test_lst), np.array(new_sanity_Y_test_lst)

In [33]:
print(np.sum(new_sanity_X_train,axis=0))
print(np.sum(out_X_train,axis=0))
print(np.sum(new_sanity_X_train-out_X_train))

print(np.sum(new_sanity_X_test,axis=0))
print(np.sum(out_X_test,axis=0))
print(np.sum(new_sanity_X_test-out_X_test))

print(np.sum(new_sanity_Y_train,axis=0))
print(np.sum(out_Y_train,axis=0))
print(np.sum(new_sanity_Y_train-out_Y_train))

print(np.sum(new_sanity_Y_test,axis=0))
print(np.sum(out_Y_test,axis=0))
print(np.sum(new_sanity_Y_test-out_Y_test))

[-157935.82646656   -7123.36459621     491.846          228.95660526
    -918.00458333     465.9669841      996.77           598.94418328
    1767.10666667    4608.            2094.01633716    1757.78356355
    6321.5           2203.99550654]
[-157935.82646656   -7123.36459621     491.846          228.95660526
    -918.00458333     465.9669841      996.77           598.94418328
    1767.10666667    4608.            2094.01633716    1757.78356355
    6321.5           2203.99550654]
0.0
[-39508.99262117  -1779.11662152    125.43           58.67836154
   -229.89691667    114.45083912    244.89          150.83782012
    443.00333333   1172.            535.88452912    441.33806128
   1587.5           551.61086601]
[-39508.99262117  -1779.11662152    125.43           58.67836154
   -229.89691667    114.45083912    244.89          150.83782012
    443.00333333   1172.            535.88452912    441.33806128
   1587.5           551.61086601]
0.0
194337.6131404116
194337.6131404116
0.0
58366.94

In [34]:
score, F1, new_sanity_FeatMat, new_sanity_Targets = \
    apply_rank_model(np.array(new_sanity_X_train_lst), np.array(new_sanity_Y_train_lst), \
                     np.array(new_sanity_X_test_lst), np.array(new_sanity_Y_test_lst), BaseNames)

(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167


In [35]:
# new_train_df, new_test_df = deepcopy(train_df), deepcopy(test_df)
# new_train_df.reindex(new_inds_train)
# new_test_df.reindex(new_inds_test)

In [36]:
new_train_df = DataFrame.from_dict(new_train_dict)
new_test_df = DataFrame.from_dict(new_test_dict)

In [37]:
new_sanity_X_train, new_sanity_Y_train, \
    new_sanity_X_test, new_sanity_Y_test = \
            new_train_df.to_numpy()[:,2:].astype('float64'), new_train_df.to_numpy()[:,1].astype('float64'), \
            new_test_df.to_numpy()[:,2:].astype('float64'), new_test_df.to_numpy()[:,1].astype('float64')

score, F1, sanity_FeatMat, sanity_Targets = \
    apply_rank_model(new_sanity_X_train, new_sanity_Y_train, new_sanity_X_test, new_sanity_Y_test, BaseNames)

(3104, 2) (777, 2)
NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)
[-1.  1.]
[-1.  1.]
Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167


## Output data

In [38]:
new_train_df.to_csv('train.csv',index=False)

In [39]:
new_test_df.to_csv('test.csv',index=False)

In [40]:
with open('KeyFeatureNames.txt','w') as fid:
    fid.write(str(KeyFeatureNames))

## Example to show how order influences the ranked matrix

In [41]:
a=np.array([[1,2,3],[4,5,6],[7,8,9]])
a1=np.array([100,200,300])
b=np.array([[4,5,6],[7,8,9],[1,2,3]])
b1=np.array([200,300,100])

print(rankedTransform(a,a1))
print(rankedTransform(b,b1))

NewFeatMat has the shape (3, 3)
NewTarget has the shape (3,)
(array([[-3, -3, -3],
       [-6, -6, -6],
       [-3, -3, -3]]), array([-1, -1, -1]))
NewFeatMat has the shape (3, 3)
NewTarget has the shape (3,)
(array([[-3, -3, -3],
       [ 3,  3,  3],
       [ 6,  6,  6]]), array([-1,  1,  1]))
